**Stateset AI Notebook**


This is going to be a working notebook of the latest developments in Deep Learning and Natural Language Processing (NLP).

In 2014 I took a course in Computational Linguistics that introduced me to logic programming in Prolog.

In 2016 I worked on my first NLP based bots using a context, utterance, entity detection and was training the model based on chatbot interactions. I used a stack powered by luis.ai and microsoft botkit, node.js while working at Apttus which later become Max. 

Fast forward 4 years and we now have BERT and Transformer based model leveraging deep learning methods and otherstate of the art natural language processing libraries.

Introductory books and resources:

https://mitpress.mit.edu/books/introduction-deep-learning

https://www.deeplearning.ai/

*    Dan Jurafsky and James H. Martin. Speech and Language Processing (3rd ed. draft)
*    Jacob Eisenstein. Natural Language Processing
*    Yoav Goldberg. A Primer on Neural Network Models for Natural Language Processing
*    Ian Goodfellow, Yoshua Bengio, and Aaron Courville. Deep Learning
*    Delip Rao and Brian McMahan. Natural Language Processing with PyTorch.
*    Programming PyTorch for Deep Learning - Ian Pointer

The other online courses I am looking through are:

Ling581-20: Advanced Computational Linguistics Arizona Spring 2020

CS224n: Natural Language Processing with Deep Learning
Stanford / Winter 2020

I will be taking this class starting in March with Jeremy Howard of fast.ai:

Data Science Institute at USF | Deep Learning I, Starting in March






The program I am going to be working on is incorporating the latest NLP technologies and libraries into Stateset. These technolgies are the following:



*   PyTorch
*   FastAI
*   BERT
*   StanfordCoreNLP
*   NLTK


The master plan:

- Build a State of the Art Natural Language Processing (NLP) engine and embed into Stateset for communicating with the platform via voice and text.
- Raise money on the Stateset platform and use that money train a better AI engine using Billions of Parameters.
- Leverage that model to create revenue to train a better AI engine and create better predictions on the global Stateset Network. 


At the same time I will be working on developing Reflektion's Shopify integration and working on implementing the Machine Learning with customers. 






The full stack consist of a next.js application which we will use the /api routing to call a nginx proxy on a flask server for the natural language processing.

This container is being served in Kubernetes Engine on GCP. 

**The Following is using fastai Tabular / Structured Data deel learning**

Tabular data is the most commonly used type of data in industry, but deep learning on tabular data receives far less attention than deep learning for computer vision and natural language processing.

0.   Data Bunch: A data format for fast ai input
1.   Dependent variable: The variable to predict
2.   Categorical columns: The text/label columns. Or the columns with low            cardinality, eg. gender, type, year etc.
3.    Continuous columns: Numeric value columns, usually with higher cardinality, eg. salary, price, temperature.
4.    Transformations: Feature engineering and handling data eg. Missing values, Normalisation etc

tabular contains all the necessary classes to deal with tabular data, across two modules:

tabular.transform: defines the TabularTransform class to help with preprocessing;

tabular.data: defines the TabularDataset that handles that data, as well as the methods to quickly get a TabularDataBunch.

**The data has been split into two groups:**

training set (train.csv)
test set (test.csv)

The training set should be used to build your machine learning models. For the training set, we provide the outcome (also known as the “ground truth”) for each passenger. Your model will be based on “features” like passengers’ gender and class. You can also use feature engineering to create new features.

The test set should be used to see how well your model performs on unseen data. For the test set, we do not provide the ground truth for each passenger. It is your job to predict these outcomes. For each passenger in the test set, use the model you trained to predict whether or not they survived the sinking of the Titanic.


In [0]:
from fastai.tabular import * 

In [0]:
path = untar_data(URLs.ADULT_SAMPLE)
path

PosixPath('/root/.fastai/data/adult_sample')

In [0]:
df = pd.read_csv(path/'adult.csv')
df.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,salary
0,49,Private,101320,Assoc-acdm,12.0,Married-civ-spouse,NaN,Wife,White,Female,0,1902,40,United-States,>=50k
1,44,Private,236746,Masters,14.0,Divorced,Exec-managerial,Not-in-family,White,Male,10520,0,45,United-States,>=50k
2,38,Private,96185,HS-grad,NaN,Divorced,NaN,Unmarried,Black,Female,0,0,32,United-States,<50k
3,38,Self-emp-inc,112847,Prof-school,15.0,Married-civ-spouse,Prof-specialty,Husband,Asian-Pac-Islander,Male,0,0,40,United-States,>=50k
4,42,Self-emp-not-inc,82297,7th-8th,NaN,Married-civ-spouse,Other-service,Wife,Black,Female,0,0,50,United-States,<50k


In [0]:
procs = [FillMissing, Categorify, Normalize]

In [0]:
valid_idx = range(len(df)-2000, len(df))

In [0]:
dep_var = 'salary'
cat_names = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native-country']

In [0]:
data = TabularDataBunch.from_df(path, df, dep_var, valid_idx=valid_idx, procs=procs, cat_names=cat_names)
print(data.train_ds.cont_names)  # `cont_names` defaults to: set(df)-set(cat_names)-{dep_var}

['capital-gain', 'fnlwgt', 'hours-per-week', 'capital-loss', 'education-num', 'age']


In [0]:
(cat_x,cont_x),y = next(iter(data.train_dl))
for o in (cat_x, cont_x, y): print(to_np(o[:5]))

[[ 6 12  3  6  1  5  2 40  1]
 [ 5 10  3  2  1  5  2 40  1]
 [ 5 12  3 15  1  5  2 40  1]
 [ 1  6  7  1  2  5  1 40  1]
 [ 3 16  3  9  1  5  2 40  1]]
[[ 1.871961  0.231205 -0.035789 -0.216787 -0.421569  0.613348]
 [ 2.54714   0.096588  0.772127 -0.216787  1.145728  2.736934]
 [-0.145922 -1.018481 -0.035789 -0.216787 -0.421569  0.173985]
 [-0.145922 -1.239652  0.287377 -0.216787 -2.380691 -0.631513]
 [-0.145922  0.920516 -0.035789 -0.216787 -0.029745  2.224345]]
[1 1 0 0 0]


In [0]:
learn = tabular_learner(data, layers=[200,100], emb_szs={'native-country': 10}, metrics=accuracy)
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,0.315658,0.320534,0.844500,00:06


In [0]:
learn.predict(df.iloc[2])

(Category <50k, tensor(0), tensor([0.9861, 0.0139]))

In [0]:
learn.predict(df.iloc[1])

(Category >=50k, tensor(1), tensor([0.3900, 0.6100]))

In [0]:
learn.predict(df.iloc[])

NameError: ignored

**The Following is using fastai Text**


In [0]:
import pathlib

In [0]:
import numpy as np

In [0]:
from fastai.text import * 

In [0]:
path = untar_data(URLs.IMDB_SAMPLE)
path

PosixPath('/root/.fastai/data/imdb_sample')

In [0]:
df = pd.read_csv(path/'texts.csv')
df.head()

,label,text,is_valid
0,negative,Un-bleeping-believable! Meg Ryan doesn't even ...,False
1,positive,This is a extremely well-made film. The acting...,False
2,negative,Every once in a long while a movie will come a...,False
3,positive,Name just says it all. I watched this movie wi...,False
4,negative,This movie succeeds at being one of the most u...,False


In [0]:
# Language model data
data_lm = TextLMDataBunch.from_csv(path, 'texts.csv')
# Classifier model data
data_clas = TextClasDataBunch.from_csv(path, 'texts.csv', vocab=data_lm.train_ds.vocab, bs=32)

In [0]:
data_lm.save('data_lm_export.pkl')
data_clas.save('data_clas_export.pkl')

In [0]:
data_lm = load_data(path, 'data_lm_export.pkl')
data_clas = load_data(path, 'data_clas_export.pkl', bs=16)

In [0]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.5)
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,4.329532,3.891484,0.285842,06:59


In [0]:
learn.unfreeze()
learn.fit_one_cycle(1, 1e-3)


epoch,train_loss,valid_loss,accuracy,time
0,3.949292,3.820232,0.293240,10:52


In [0]:
learn.predict("This is a review about", n_words=10)

'This is a review about New York City from records directed by'

In [0]:
learn.save_encoder('ft_enc')

In [0]:
learn = text_classifier_learner(data_clas, AWD_LSTM, drop_mult=0.5)
learn.load_encoder('ft_enc')


In [0]:
data_clas.show_batch()

text,target
"xxbos xxmaj xxunk xxmaj victor xxmaj xxunk : a xxmaj review \n \n xxmaj you know , xxmaj xxunk xxmaj victor xxmaj xxunk is like sticking your hands into a big , xxunk xxunk of xxunk . xxmaj it 's warm and xxunk , but you 're not sure if it feels right . xxmaj try as i might , no matter how warm and xxunk xxmaj xxunk xxmaj",negative
"xxbos xxup the xxup shop xxup around xxup the xxup corner is one of the sweetest and most feel - good romantic comedies ever made . xxmaj there 's just no getting around that , and it 's hard to actually put one 's feeling for this film into words . xxmaj it 's not one of those films that tries too hard , nor does it come up with",positive
"xxbos xxmaj now that xxmaj che(2008 ) has finished its relatively short xxmaj australian cinema run ( extremely limited xxunk screen in xxmaj sydney , after xxunk ) , i can xxunk join both xxunk of "" xxmaj at xxmaj the xxmaj movies "" in taking xxmaj steven xxmaj soderbergh to task . \n \n xxmaj it 's usually satisfying to watch a film director change his style /",negative
"xxbos xxmaj this film sat on my xxmaj tivo for weeks before i watched it . i dreaded a self - indulgent xxunk flick about relationships gone bad . i was wrong ; this was an xxunk xxunk into the screwed - up xxunk of xxmaj new xxmaj yorkers . \n \n xxmaj the format is the same as xxmaj max xxmaj xxunk ' "" xxmaj la xxmaj ronde",positive
"xxbos xxmaj many neglect that this is n't just a classic due to the fact that it 's the first xxup 3d game , or even the first xxunk - up . xxmaj it 's also one of the first stealth games , one of the xxunk definitely the first ) truly claustrophobic games , and just a pretty well - xxunk gaming experience in general . xxmaj with graphics",positive


In [0]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(5e-3/2., 5e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.578390,0.473341,0.810945,07:59


In [0]:
learn.unfreeze()
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

In [0]:
learn.predict("This was a great movie!")

(Category positive, tensor(1), tensor([0.3372, 0.6628]))

In [0]:
learn.predict("That movie was not that good")

(Category negative, tensor(0), tensor([0.7610, 0.2390]))

In [0]:
learn.predict("It was ok")

(Category negative, tensor(0), tensor([0.5557, 0.4443]))

In [0]:
learn.predict("That movie was epic!")

(Category positive, tensor(1), tensor([0.4491, 0.5509]))

In [0]:
learn.predict("Awesome movie!")

(Category negative, tensor(0), tensor([0.5094, 0.4906]))

This was my first model using fastai with NLP built on 1/22/2020 by Dom Steil


In [0]:
learn.save('nlp-8')

Recurrent Neural Network (RNN) for Natural Language Processing using three libraries:



*   torchtext
*   spaCy
*   pandas






In [0]:
import pandas as pd
tweetsDF = pd.read_csv("training.1600000.processed.noemoticon.csv",
                       engine="python", header=None)

In [0]:
from torchtext import data


LABEL = data.LabelField()
STATESET = data.Field(tokenize='spacy', lower=true)

NameError: ignored

In [0]:
fields = ['id', 'product_id', 'name', 'description', 'product_url', 'image_url', 'breadcrumbs', 'price', 'special_price', 'status', 'quantity', 'inventory_status', 'custom_color' ];

Define the fields and product a list that maps them onto the list of rows that are in the CSV file produced from the Product Data Feed.

In [0]:
statesetDataset = torchtext.data.TabularDataset(
    path="./training-processed.csv",
    format="CSV",
    fields=fields,
    skip_header=False
)

NameError: ignored

Split the training data into training, testing and validation sets

In [0]:
(train, test, valid) = statesetDataset.split(split_ratio=[0.8,0.1,0.1])

(len(train),len(test),len(valid))

NameError: ignored

In [0]:
vocab_size = 2000
STATESET.build_vocab(train, max_size = vocab_size)

NameError: ignored

In [0]:
train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train, valid, test),
    batch_size = 32,
    device = device)

SyntaxError: ignored